In [1]:
import folium
import openrouteservice
import json

def buat_peta_isochrone_multi(api_key, koordinat, profile, list_menit, nama_file="isochrone_multi.html"):
    """
    Buat peta isochrone untuk beberapa waktu sekaligus.

    Parameters:
    - api_key (str): ORS API key (langsung sebagai argumen)
    - koordinat (list): [lon, lat]
    - profile (str): 'foot-walking', 'driving-car', dll
    - list_menit (list): daftar waktu dalam menit [5, 10, 15]
    - nama_file (str): nama file HTML output
    """
    client = openrouteservice.Client(key=api_key)

    # Konversi menit ke detik
    range_detik = [m * 60 for m in list_menit]

    # Ambil interval default (selisih antar menit)
    interval = (list_menit[1] - list_menit[0]) * 60 if len(list_menit) > 1 else None

    # Panggil API
    isochrones = client.isochrones(
        locations=[koordinat],
        profile=profile,
        range=range_detik,
        interval=interval,
        attributes=["area"]
    )

    # Buat peta dasar
    m = folium.Map(location=koordinat[::-1], zoom_start=13)

    # Warna untuk tiap zona
    warna = ['#1a9641', '#a6d96a', '#fdae61', '#d7191c', '#2b83ba']

    for i, feature in enumerate(isochrones['features']):
        waktu = feature['properties']['value'] // 60
        warna_fill = warna[i % len(warna)]
        folium.GeoJson(
            feature,
            name=f"{waktu} menit",
            style_function=lambda feature, clr=warna_fill: {
                'fillColor': clr,
                'color': clr,
                'weight': 1,
                'fillOpacity': 0.4,
            },
            tooltip=f"{waktu} menit dari titik pusat"
        ).add_to(m)

    folium.Marker(location=koordinat[::-1], tooltip="Titik Pusat").add_to(m)
    folium.LayerControl().add_to(m)

    # Simpan file HTML dan GeoJSON
    m.save(nama_file)
    with open("isochrone_output.geojson", "w") as f:
        json.dump(isochrones, f)

    print(f"✅ Peta disimpan ke: {nama_file}")
    print("📁 GeoJSON disimpan ke: isochrone_output.geojson")


In [2]:
import os
api_key = os.getenv("ORS_API_KEY")

if __name__ == "__main__":
    # Langsung isi di sini
    API_KEY = api_key
    KOORDINAT = [106.827153, -6.175392]  # Monas
    PROFILE = "foot-walking"
    RANGE_MENIT = [5, 10, 15]

    buat_peta_isochrone_multi(API_KEY, KOORDINAT, PROFILE, RANGE_MENIT)


✅ Peta disimpan ke: isochrone_multi.html
📁 GeoJSON disimpan ke: isochrone_output.geojson


In [3]:
import folium
import openrouteservice
import json
import os

# Profil yang didukung oleh ORS
PROFILE_LIST = [
    "driving-car",
    "driving-hgv",
    "cycling-regular",
    "cycling-electric",
    "foot-walking",
    "foot-hiking",
    "wheelchair"
]

# Warna default untuk visualisasi profile (bisa dikustomisasi)
COLOR_MAP = {
    "driving-car": "#1a9641",
    "driving-hgv": "#a6d96a",
    "cycling-regular": "#fdae61",
    "cycling-electric": "#d7191c",
    "foot-walking": "#2b83ba",
    "foot-hiking": "#984ea3",
    "wheelchair": "#ff7f00"
}

def ambil_isochrone(client, koordinat, profile, range_detik):
    """Panggil ORS isochrone API untuk 1 profile."""
    return client.isochrones(
        locations=[koordinat],
        profile=profile,
        range=range_detik,
        attributes=["area"]
    )

def buat_peta_gabungan(api_key, koordinat, list_menit, folder_output="output"):
    """Gabungkan semua profile dalam satu peta HTML dengan masing-masing layer isochrone."""

    os.makedirs(folder_output, exist_ok=True)
    m = folium.Map(location=koordinat[::-1], zoom_start=13)
    client = openrouteservice.Client(key=api_key)
    range_detik = [m * 60 for m in list_menit]

    for profile in PROFILE_LIST:
        try:
            print(f"🔄 Mengambil isochrone untuk: {profile}")
            isochrones = ambil_isochrone(client, koordinat, profile, range_detik)

            warna_fill = COLOR_MAP.get(profile, "#666666")

            for i, feature in enumerate(isochrones["features"]):
                waktu = feature['properties']['value'] // 60
                folium.GeoJson(
                    feature,
                    name=f"{waktu} menit - {profile}",
                    style_function=lambda feature, clr=warna_fill: {
                        'fillColor': clr,
                        'color': clr,
                        'weight': 1,
                        'fillOpacity': 0.4,
                    },
                    tooltip=f"{waktu} menit - {profile}"
                ).add_to(m)

            # Simpan GeoJSON
            geojson_path = os.path.join(folder_output, f"{profile.replace('-', '_')}_isochrone.geojson")
            with open(geojson_path, "w") as f:
                json.dump(isochrones, f)
            print(f"✅ GeoJSON disimpan: {geojson_path}")

        except Exception as e:
            print(f"❌ Gagal untuk profile {profile}: {e}")

    # Tambahkan titik pusat
    folium.Marker(location=koordinat[::-1], tooltip="Titik Pusat").add_to(m)
    folium.LayerControl().add_to(m)

    # Simpan HTML
    html_path = os.path.join(folder_output, "semua_profile_isochrone.html")
    m.save(html_path)
    print(f"📍 Peta gabungan disimpan: {html_path}")


In [5]:
import os
api_key = os.getenv("ORS_API_KEY")

if __name__ == "__main__":
    # Langsung isi di sini
    API_KEY = api_key
    KOORDINAT = [106.827153, -6.175392]  # Monas
    RANGE_MENIT = [5, 10, 15]

    buat_peta_gabungan(API_KEY, KOORDINAT, RANGE_MENIT)


🔄 Mengambil isochrone untuk: driving-car
✅ GeoJSON disimpan: output\driving_car_isochrone.geojson
🔄 Mengambil isochrone untuk: driving-hgv
✅ GeoJSON disimpan: output\driving_hgv_isochrone.geojson
🔄 Mengambil isochrone untuk: cycling-regular
✅ GeoJSON disimpan: output\cycling_regular_isochrone.geojson
🔄 Mengambil isochrone untuk: cycling-electric
✅ GeoJSON disimpan: output\cycling_electric_isochrone.geojson
🔄 Mengambil isochrone untuk: foot-walking
✅ GeoJSON disimpan: output\foot_walking_isochrone.geojson
🔄 Mengambil isochrone untuk: foot-hiking
✅ GeoJSON disimpan: output\foot_hiking_isochrone.geojson
🔄 Mengambil isochrone untuk: wheelchair
✅ GeoJSON disimpan: output\wheelchair_isochrone.geojson
📍 Peta gabungan disimpan: output\semua_profile_isochrone.html


In [1]:
import os
import json
import folium
import geopandas as gpd
import openrouteservice
from shapely.geometry import shape

# Daftar profile dan warna
PROFILE_LIST = [
    "driving-car",
    "driving-hgv",
    "cycling-regular",
    "cycling-electric",
    "foot-walking",
    "foot-hiking",
    "wheelchair"
]

COLOR_MAP = {
    "driving-car": "#1a9641",
    "driving-hgv": "#a6d96a",
    "cycling-regular": "#fdae61",
    "cycling-electric": "#d7191c",
    "foot-walking": "#2b83ba",
    "foot-hiking": "#984ea3",
    "wheelchair": "#ff7f00"
}

def ambil_centroid_dari_kml(file_kml):
    """Ambil titik centroid dari file .kml"""
    gdf = gpd.read_file(file_kml, driver='KML')
    if gdf.empty:
        raise ValueError("File KML kosong atau tidak valid.")
    centroid = gdf.unary_union.centroid
    return [centroid.x, centroid.y]  # [lon, lat]

def buat_peta_gabungan_dari_kml(api_key, file_kml, list_menit, folder_output="output"):
    os.makedirs(folder_output, exist_ok=True)

    koordinat = ambil_centroid_dari_kml(file_kml)
    print(f"📍 Centroid dari file KML: {koordinat[::-1]}")

    m = folium.Map(location=koordinat[::-1], zoom_start=13)
    client = openrouteservice.Client(key=api_key)
    range_detik = [m * 60 for m in list_menit]

    area_summary = []

    for profile in PROFILE_LIST:
        try:
            print(f"🔄 Mengambil isochrone untuk: {profile}")
            isochrones = client.isochrones(
                locations=[koordinat],
                profile=profile,
                range=range_detik,
                attributes=["area"]
            )

            warna_fill = COLOR_MAP.get(profile, "#666666")
            total_area_m2 = 0

            for i, feature in enumerate(isochrones["features"]):
                waktu = feature['properties']['value'] // 60
                area_m2 = feature["properties"].get("area", 0)
                total_area_m2 += area_m2

                folium.GeoJson(
                    feature,
                    name=f"{waktu} menit - {profile}",
                    style_function=lambda feature, clr=warna_fill: {
                        'fillColor': clr,
                        'color': clr,
                        'weight': 1,
                        'fillOpacity': 0.4,
                    },
                    tooltip=f"{waktu} menit - {profile} ({round(area_m2/1000000, 2)} km²)"
                ).add_to(m)

            # Simpan GeoJSON
            geojson_path = os.path.join(folder_output, f"{profile.replace('-', '_')}_isochrone.geojson")
            with open(geojson_path, "w") as f:
                json.dump(isochrones, f)
            print(f"✅ GeoJSON disimpan: {geojson_path}")

            area_summary.append({
                "profile": profile,
                "total_area_m2": round(total_area_m2, 2),
                "total_area_km2": round(total_area_m2 / 1e6, 2)
            })

        except Exception as e:
            print(f"❌ Gagal untuk profile {profile}: {e}")

    folium.Marker(location=koordinat[::-1], tooltip="Titik Pusat").add_to(m)
    folium.LayerControl().add_to(m)

    html_path = os.path.join(folder_output, "semua_profile_isochrone.html")
    m.save(html_path)
    print(f"🌐 Peta gabungan disimpan: {html_path}")

    json_path = os.path.join(folder_output, "ringkasan_area.json")
    with open(json_path, "w") as f:
        json.dump(area_summary, f, indent=2)

    print(f"📊 Ringkasan area disimpan: {json_path}")
    print("🧾 Area total per profile:")
    for a in area_summary:
        print(f"   - {a['profile']:>15}: {a['total_area_km2']} km²")


In [3]:
import os

if __name__ == "__main__":
    API_KEY = os.getenv("ORS_API_KEY")  # Pastikan API_KEY sudah di-set di environment variable
    FILE_KML = "BSD CITY2.kml"  # Ganti dengan file KML kamu
    RANGE_MENIT = [15, 30, 45]

    buat_peta_gabungan_dari_kml(API_KEY, FILE_KML, RANGE_MENIT)


C:\Users\IABEE - Benno\AppData\Local\Temp\ipykernel_177716\1784884276.py:34: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = gdf.unary_union.centroid


📍 Centroid dari file KML: [-6.3069927026513115, 106.65096429416006]
🔄 Mengambil isochrone untuk: driving-car
✅ GeoJSON disimpan: output\driving_car_isochrone.geojson
🔄 Mengambil isochrone untuk: driving-hgv
✅ GeoJSON disimpan: output\driving_hgv_isochrone.geojson
🔄 Mengambil isochrone untuk: cycling-regular
✅ GeoJSON disimpan: output\cycling_regular_isochrone.geojson
🔄 Mengambil isochrone untuk: cycling-electric
✅ GeoJSON disimpan: output\cycling_electric_isochrone.geojson
🔄 Mengambil isochrone untuk: foot-walking
✅ GeoJSON disimpan: output\foot_walking_isochrone.geojson
🔄 Mengambil isochrone untuk: foot-hiking
✅ GeoJSON disimpan: output\foot_hiking_isochrone.geojson
🔄 Mengambil isochrone untuk: wheelchair
✅ GeoJSON disimpan: output\wheelchair_isochrone.geojson
🌐 Peta gabungan disimpan: output\semua_profile_isochrone.html
📊 Ringkasan area disimpan: output\ringkasan_area.json
🧾 Area total per profile:
   -     driving-car: 4011.55 km²
   -     driving-hgv: 3496.72 km²
   - cycling-regul

In [1]:
import os
import json
import time
import folium
import geopandas as gpd
import openrouteservice
from shapely.geometry import shape

# Daftar profile dan warna
PROFILE_LIST = [
    "driving-car",
    "cycling-regular",
    "foot-walking"
]

COLOR_MAP = {
    "driving-car": "#1a9641",
    "driving-hgv": "#a6d96a",
    "cycling-regular": "#fdae61",
    "cycling-electric": "#d7191c",
    "foot-walking": "#2b83ba",
    "foot-hiking": "#984ea3",
    "wheelchair": "#ff7f00"
}

def retry_request(func, max_retries=3, wait=5):
    for attempt in range(max_retries):
        try:
            return func()
        except openrouteservice.exceptions.ApiError as e:
            print(f"⚠️ Rate limit: mencoba ulang ke-{attempt+1} dalam {wait} detik...")
            time.sleep(wait)
    raise RuntimeError("❌ Gagal setelah beberapa kali mencoba ulang.")

def buat_peta_gabungan_dari_kml(api_key, file_kml, list_menit, folder_output="output", sampling_interval=20):
    os.makedirs(folder_output, exist_ok=True)

    # === [1] Baca KML dan ambil semua titik batas polygon
    gdf_kml = gpd.read_file(file_kml, driver='KML')
    if gdf_kml.empty:
        raise ValueError("File KML kosong atau tidak valid.")

    boundary_points = []
    for geom in gdf_kml.geometry:
        if geom.geom_type == "Polygon":
            boundary_points.extend(list(geom.exterior.coords))
        elif geom.geom_type == "MultiPolygon":
            for poly in geom.geoms:
                boundary_points.extend(list(poly.exterior.coords))

    koordinat_list = [[point[0], point[1]] for point in boundary_points][::sampling_interval]  # sampling
    print(f"\U0001F4CD Menggunakan {len(koordinat_list)} titik dari batas polygon KML (dengan sampling)")

    # === [2] Buat peta dasar
    centroid = gdf_kml.unary_union.centroid
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=13, control_scale=True)
    client = openrouteservice.Client(key=api_key)
    range_detik = [m * 60 for m in list_menit]
    area_summary = []

    # === [3] Tambahkan polygon asli dari file KML jika valid
    try:
        geom_type = gdf_kml.geometry.geom_type.unique()
        print("Tipe geometrinya:", geom_type)
        if any(gt in ["Polygon", "MultiPolygon"] for gt in geom_type):
            folium.GeoJson(
                gdf_kml,
                name="Wilayah Asli (KML)",
                overlay=True,
                control=True,
                style_function=lambda feature: {
                    'fillColor': '#ffc0cb',
                    'color': '#ff69b4',
                    'weight': 2,
                    'fillOpacity': 1.0
                },
                tooltip="Polygon Asli dari KML"
            ).add_to(m)

            bounds = gdf_kml.total_bounds
            m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
        else:
            print("⚠️ File KML tidak mengandung Polygon.")
    except Exception as e:
        print(f"❌ Gagal menambahkan polygon asli: {e}")

    # === [4] Tambahkan isochrone dari semua profile untuk setiap titik di batas polygon
    for profile in PROFILE_LIST:
        try:
            print(f"🔄 Mengambil isochrone untuk: {profile}")
            warna_fill = COLOR_MAP.get(profile, "#666666")
            total_area_m2 = 0

            for idx, koordinat in enumerate(koordinat_list):
                try:
                    isochrones = retry_request(lambda: client.isochrones(
                        locations=[koordinat],
                        profile=profile,
                        range=range_detik,
                        attributes=["area"]
                    ))

                    for i, feature in enumerate(isochrones["features"]):
                        waktu = feature['properties']['value'] // 60
                        area_m2 = feature["properties"].get("area", 0)
                        total_area_m2 += area_m2

                        folium.GeoJson(
                            feature,
                            name=f"{waktu} menit - {profile}",
                            style_function=lambda feature, clr=warna_fill: {
                                'fillColor': clr,
                                'color': clr,
                                'weight': 1,
                                'fillOpacity': 0.2,
                            },
                            tooltip=f"{waktu} menit - {profile} ({round(area_m2/1000000, 2)} km²)"
                        ).add_to(m)

                    time.sleep(2.5)  # jeda antar request ke API

                except Exception as e:
                    print(f"❌ Gagal untuk titik ke-{idx}: {e}")
                    continue

            geojson_path = os.path.join(folder_output, f"{profile.replace('-', '_')}_isochrone.geojson")
            with open(geojson_path, "w") as f:
                json.dump(isochrones, f)

            area_summary.append({
                "profile": profile,
                "total_area_m2": round(total_area_m2, 2),
                "total_area_km2": round(total_area_m2 / 1e6, 2)
            })

        except Exception as e:
            print(f"❌ Gagal untuk profile {profile}: {e}")

    # === [5] Tambahkan titik centroid dan layer control
    folium.Marker(location=[centroid.y, centroid.x], tooltip="Titik Pusat").add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)

    # === [6] Simpan semua output
    html_path = os.path.join(folder_output, "semua_profile_isochrone3.html")
    m.save(html_path)
    print(f"🌐 Peta gabungan disimpan: {html_path}")

    json_path = os.path.join(folder_output, "ringkasan_area.json")
    with open(json_path, "w") as f:
        json.dump(area_summary, f, indent=2)

    print(f"📊 Ringkasan area disimpan: {json_path}")
    print("🧾 Area total per profile:")
    for a in area_summary:
        print(f"   - {a['profile']:>15}: {a['total_area_km2']} km²")


ModuleNotFoundError: No module named 'folium'

In [21]:
import os
api_key = os.getenv("ORS_API_KEY2")  # Pastikan API_KEY sudah di
api_key

'eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImIyNTRhMjYwZDEwNDQ1YmE4ZTgxZjUzMTJiOWY5ODdjIiwiaCI6Im11cm11cjY0In0='

In [28]:
buat_peta_gabungan_dari_kml(api_key, "BSD CITY2.kml", [15, 30, 45])


📍 Menggunakan 16 titik dari batas polygon KML (dengan sampling)
Tipe geometrinya: ['Polygon']
🔄 Mengambil isochrone untuk: driving-car


C:\Users\IABEE - Benno\AppData\Local\Temp\ipykernel_222060\3937077304.py:55: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = gdf_kml.unary_union.centroid


🔄 Mengambil isochrone untuk: cycling-regular
🔄 Mengambil isochrone untuk: foot-walking


d:\kerjaan\AI\venv\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
d:\kerjaan\AI\venv\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
d:\kerjaan\AI\venv\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
d:\kerjaan\AI\venv\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
d:\kerjaan\AI\venv\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceede

🌐 Peta gabungan disimpan: output\semua_profile_isochrone3.html
📊 Ringkasan area disimpan: output\ringkasan_area.json
🧾 Area total per profile:
   -     driving-car: 68237.93 km²
   - cycling-regular: 7832.35 km²
   -    foot-walking: 571.97 km²
